#Problem

# Puzzle-and-Dragon-Max-Combos
An exercise trying to find the max. combos on 6*5 grid

Given a 6 * 5 grid G that every index contains a random element from 6 different type (say A,B,C,D,E,F), there will be N waves to destroy the elements, until no more elements can be destroyed. 

The elements can only be destroyed when there appears with 3 or more consecutive elements from same type horizontally or vertically. Matches with more than 3 elements that in same type will be considered as 1 combo. For instance, a 2 x 3 clearance / a horizontal column of 3 and a vertical row of 3 that are connected by an element in the middle, will be considered as 1 combo. The system will scan through the grid from (0,0) to (4,5) on row-first basis, i.e. (0,0) -> (0,1) -> (0,2) -> …  -> (0,5) -> (1,0) -> (1,1) -> … -> (4,5) for 1 wave. After each wave, the remaining elements will be dropped vertically and start the next wave, no elements will be filled after each wave. 

With given G, player can rearrange the elements with any possible arrangement for initial round, but the total number of each element types must remain the same after the arrangement. 

Example flow, 

1. 0 combos

AAACDF

BDCCEF

BDEEEE

BBBCDF

CDAAAF

2. 4 combos

AAACDF

BDCCEF

BDEEEE

BBBCDF

CDAAAF


XXXXXX

XXXXXF

XDXCDF

XDXCEF

CDCCDF

3. 4 + 3, 7 combos total

XXXXXX

XXXXXF

XDXCDF

XDXCEF

CDCCDF


XXXXXX

XXXXXX

XXXXDX

XXXXEX

CXCXDX

The goal is to find the best initial arrangement in order to get highest possible number of combos with either approach from below:

	1. Design an algorithm to find the arrangement .
	2. Use Machine Learning to train the model to find the arrangement.

*Reference: http://pad.wikia.com/wiki/Game_Mechanics

In [ ]:
import numpy as np
import itertools


In [ ]:
def create_empty_grid():
    return np.full((5, 6), -1)


def find_score(grid):
    matches = []
    for x in range(0, 5):
        for y in range(0, 6):
            e1 = grid[x][y]
            # vertical (orientation = 0)
            if x < 3:
                e2 = grid[x + 1][y]
                e3 = grid[x + 2][y]
                if e1 == e2 and e1 == e3:
                    matches.append((e1, x, y, 0))
            # horizontal (orientation = 1)
            if y < 4:
                e2 = grid[x][y + 1]
                e3 = grid[x][y + 2]
                if e1 == e2 and e1 == e3:
                    matches.append((e1, x, y, 1))

    match_grid = create_empty_grid()
    score = 0
    for (e_type, x, y, orientation) in matches:
        for i in range(0, 3):
            is_in_match = False
            if orientation == 0:
                e1 = match_grid[x][y]
                e2 = match_grid[x + 1][y]
                e3 = match_grid[x + 2][y]
                if e1 == -1:
                    match_grid[x][y] = e_type
                else:
                    is_in_match = True
                if e2 == -1:
                    match_grid[x + 1][y] = e_type
                else:
                    is_in_match = True
                if e3 == -1:
                    match_grid[x + 2][y] = e_type
                else:
                    is_in_match = True

                if not is_in_match:
                    score = score + 1
            else:
                e1 = match_grid[x][y]
                e2 = match_grid[x][y + 1]
                e3 = match_grid[x][y + 2]
                if e1 == -1:
                    match_grid[x][y] = e_type
                else:
                    is_in_match = True
                if e2 == -1:
                    match_grid[x][y + 1] = e_type
                else:
                    is_in_match = True
                if e3 == -1:
                    match_grid[x][y + 2] = e_type
                else:
                    is_in_match = True

                if not is_in_match:
                    score = score + 1

    print(matches)
    print(score)
    return score

In [ ]:
def get_element_numbers(grid):
    numbers = [0, 0, 0, 0, 0, 0]
    for x in range(0, 5):
        for y in range(0, 6):
            element = grid[x][y]
            numbers[element] = numbers[element] + 1

    return numbers


def find_score_in_simple_way(grid):
    numbers = [n / 3 for n in get_element_numbers(grid)]
    return sum(numbers)

#Analyzing the problem

With above requirements, we can easily found that the max. combos for each grid will only be 10. Because the will only be 6 * 5 = 30 of elements, and in most ideal case we can destroy the elements with 3 consecutive elements, which is equal to 30 / 3 = 10. 

From here we can further simplify the problem. Given that we can relocate the elements with any arrangement we want, the initial arrangement can’t provide useful information for us now. All we need to know is the number of different elements. Take above pattern as an example, we can simplify it to a 1-D array A: {6, 5, 5, 5, 5, 4}, which equals to the total numbers of the array {A, B, C, D, E, F}. Since there are no possible ways to achieve higher combos with insufficient elements, we can simply find the sum of A % 3, which is 2 + 1 + 1 + 1 + 1 + 1 = 7 in this case. 

Note that above algorithm is only suitable when it happens to be a relatively even distribution. When a single element has a total number = 15, it reaches its maximum combos, which is 5. See below as an example,

AXAXAA

AXAXAA

AXAXAA

XXXXXX

AAAXXX

When we try to add A on the into the grid, it will not increase the no, of combos but with a possibility to connect the matches and hence reduce the number of combos. 


Base on this, we build a simple method to test the score of the grid, named "find_score_in_simple_way".

However, when number of A >= 21, there will only be 9 or less free spaces for us to insert the A, that will not be possible to not connect any existing matches. Therefore, starting from A = 21, the combos will decrease with the rise of number of A.

Number of A = 20, example grid:

AXAXAA

AXAXAA

AXAXAA

XAXAXX

AAAAAA

Therefore, we used find_score method to identify connected elements and remove duplicated matches fomr the grid. 


After setting the score method, we now want to find the possible inital arrangement for the board. To obtain this goal, we have serval methods. 

1. Brute-force all the possible permutation of 5 * 6 gird for 6 elements. In this case, it will probably take forever to load since the possible number of permutation would be 6^30 = 2.210739197+e23.

2. Method in 1 contains many duplicated and unnecessary arrangements, which the method is very inefficient and unnecessary. As we know the number of each elements are fixed and the elements are identical with in same group. we can find the permutation to be : $\prod_0^{30} n$ for n being the number of distinct elements. For simple case like A=24, B=6, C=0, D=0, E=0, F=0, the permutation would be 2^24 * 1^6 = 16,777,216. We can use a Depth-First search to implement it.




In [ ]:
def dfs(nums, used, lst, res):
    if len(lst) == len(nums):
        print lst
        res.append(lst)
        return

    for i in range(0, len(nums)):
        if used[i]:
            continue
        if i > 0 and nums[i - 1] == nums[i] and not used[i - 1]:
            continue
        used[i] = True
        lst[i].append(nums[i])
        dfs(nums, used, lst, res)
        used[i] = False
        lst.remove(len(lst) - 1)

def test1(nums):
    res = [[]]
    used = [False for i in range(30)]
    lst = [0 for i in range(30)]
    dfs(nums, used,lst,res)